In [7]:
from typing import TypedDict, Literal

class PortfolioState(TypedDict):
    amount_usd: float
    total_usd: float
    target_currency: Literal["INR", "EUR"]
    total: float

In [8]:
def calc_total(state: PortfolioState) -> PortfolioState:
    """
    Calculate the total value in USD.
    """
    state['total_usd'] = state['amount_usd'] * 1.08
    return state

In [9]:
def convert_to_inr(state: PortfolioState) -> PortfolioState:
    """
    Convert the total value to INR.
    """
    state['total'] = state['total_usd'] * 85
    return state

In [10]:
def convert_to_eur(state: PortfolioState) -> PortfolioState:
    """
    Convert the total value to EUR.
    """
    state['total'] = state['total_usd'] * 0.9
    return state

In [11]:
def choose_conversion(state: PortfolioState) -> str:
    """
    Choose the conversion function based on the target currency.
    """
    return state['target_currency']


In [12]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(PortfolioState)

builder.add_node("calc_total", calc_total)
builder.add_node("convert_to_inr", convert_to_inr)
builder.add_node("convert_to_eur", convert_to_eur)

builder.add_edge(START, "calc_total")
builder.add_conditional_edges(
    "calc_total",
    choose_conversion,
    {
        "INR": "convert_to_inr",
        "EUR": "convert_to_eur"
    }
)
builder.add_edge("convert_to_inr", END)
builder.add_edge("convert_to_eur", END)

graph = builder.compile()

In [14]:
graph.invoke({"amount_usd": 1000, "target_currency": "EUR"})

{'amount_usd': 1000,
 'total_usd': 1080.0,
 'target_currency': 'EUR',
 'total': 972.0}